In [1]:
# !pip install -q transformers datasets scikit-learn accelerate torch pandas

import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import os

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed,
)
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight



SEED = 42
MAX_LEN = 512
BATCH_SIZE = 4
GRAD_ACCUMULATION = 4
LR = 8e-6
EPOCHS = 15

DEBERTA_NAME = "microsoft/deberta-v3-large"
ROBERTA_NAME = "roberta-large"

DEBERTA_OUTPUT_DIR = "./deberta_task1_single"
ROBERTA_OUTPUT_DIR = "./roberta_task1_single"

ENSEMBLE_NAME = "deberta_roberta_task1_ensemble"


# Reproducibility

def set_deterministic(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_deterministic(SEED)
set_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"


# Load and preprocess QEvasion (Task 1: clarity_label)

dataset = load_dataset("ailsntua/QEvasion")

def preprocess(example):
    # Same pattern as your DeBERTa script
    text = f"Question: {example['question']} Answer: {example['interview_answer']}"
    return {"text": text, "clarity_label": example["clarity_label"]}

full_train = dataset["train"].map(preprocess)
held_out_test_ds = dataset["test"].map(preprocess)

# Encode Task 1 labels (3-way clarity)
full_train = full_train.class_encode_column("clarity_label")
held_out_test_ds = held_out_test_ds.class_encode_column("clarity_label")

# Stratified split for train/dev
split = full_train.train_test_split(
    test_size=0.1,
    seed=SEED,
    stratify_by_column="clarity_label",
)
train_ds = split["train"]
eval_ds = split["test"]

print(f"Dataset Sizes: Train: {len(train_ds)}, Dev: {len(eval_ds)}, Test: {len(held_out_test_ds)}")

labels = train_ds.features["clarity_label"].names
label2id = {name: i for i, name in enumerate(labels)}
id2label = {i: name for name, i in label2id.items()}
print(f"Task 1 Labels: {labels}")

# Class weights for imbalance
y_train = train_ds["clarity_label"]
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train,
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print("Class weights:", class_weights)


# Metrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": (preds == labels).mean(),
        "macro_f1": f1_score(labels, preds, average="macro"),
    }


# Custom Trainer with class weights

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(
            weight=class_weights_tensor,
            label_smoothing=0.1,
        )
        loss = loss_fct(
            logits.view(-1, model.config.num_labels),
            labels.view(-1),
        )
        return (loss, outputs) if return_outputs else loss


# Helper: tokenization per model

def tokenize_dataset_for_model(model_name, train_ds, eval_ds, test_ds):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_fn(batch):
        return tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
        )

    tokenized_train = train_ds.map(tokenize_fn, batched=True)
    tokenized_eval = eval_ds.map(tokenize_fn, batched=True)
    tokenized_test = test_ds.map(tokenize_fn, batched=True)

    # Map clarity_label -> labels
    tokenized_train = tokenized_train.map(lambda x: {"labels": x["clarity_label"]})
    tokenized_eval = tokenized_eval.map(lambda x: {"labels": x["clarity_label"]})
    tokenized_test = tokenized_test.map(lambda x: {"labels": x["clarity_label"]})

    # Set format for PyTorch
    cols = ["input_ids", "attention_mask", "labels"]
    if "token_type_ids" in tokenized_train.column_names:
        cols.append("token_type_ids")

    tokenized_train.set_format(type="torch", columns=cols)
    tokenized_eval.set_format(type="torch", columns=cols)
    tokenized_test.set_format(type="torch", columns=cols)

    return tokenized_train, tokenized_eval, tokenized_test


# Helper: train a single model

def train_single_model(model_name, output_dir):
    print(f"\n==== Training {model_name} on Task 1 (clarity) ====")

    tokenized_train, tokenized_eval, tokenized_test = tokenize_dataset_for_model(
        model_name, train_ds, eval_ds, held_out_test_ds
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id,
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=LR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE * 2,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        num_train_epochs=EPOCHS,
        weight_decay=0.05,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        report_to="none",
        seed=SEED,
    )

    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
    )

    trainer.train()

    print(f"\nEvaluating {model_name} on held-out Task 1 test set...")
    test_results = trainer.evaluate(tokenized_test)
    print(f"{model_name} Test macro-F1:", round(test_results["eval_macro_f1"], 4))
    print(f"{model_name} Test accuracy:", round(test_results["eval_accuracy"], 4))
    print(f"{model_name} Test loss:", round(test_results["eval_loss"], 4))

    return trainer, tokenized_test


# Train RoBERTa-large and DeBERTa-v3-large

roberta_trainer, roberta_test = train_single_model(ROBERTA_NAME, ROBERTA_OUTPUT_DIR)
deberta_trainer, deberta_test = train_single_model(DEBERTA_NAME, DEBERTA_OUTPUT_DIR)


# Ensemble: average logits from both models on the test set

print("\n==== Building RoBERTa + DeBERTa Ensemble on Task 1 test set ====")

# Get raw logits (and labels) from both models
roberta_preds = roberta_trainer.predict(roberta_test)
deberta_preds = deberta_trainer.predict(deberta_test)

roberta_logits = roberta_preds.predictions  # shape: (N, num_labels)
deberta_logits = deberta_preds.predictions  # shape: (N, num_labels)
labels_np = roberta_preds.label_ids         # same as deberta_preds.label_ids

# Sanity check
assert roberta_logits.shape == deberta_logits.shape, "Logit shapes differ between models!"
assert np.array_equal(labels_np, deberta_preds.label_ids), "Label order mismatch!"

# Simple ensemble: average logits
ensemble_logits = (roberta_logits + deberta_logits) / 2.0
ensemble_preds = np.argmax(ensemble_logits, axis=-1)

ensemble_accuracy = (ensemble_preds == labels_np).mean()
ensemble_macro_f1 = f1_score(labels_np, ensemble_preds, average="macro")

print(f"Ensemble (RoBERTa + DeBERTa) Test accuracy: {ensemble_accuracy:.4f}")
print(f"Ensemble (RoBERTa + DeBERTa) Test macro-F1: {ensemble_macro_f1:.4f}")

# Optionally, write out predictions to CSV for analysis
os.makedirs(ENSEMBLE_NAME, exist_ok=True)
out_path = os.path.join(ENSEMBLE_NAME, "task1_ensemble_predictions.csv")

# Map back to label strings
id2label_str = id2label
pred_label_str = [id2label_str[int(i)] for i in ensemble_preds]
true_label_str = [id2label_str[int(i)] for i in labels_np]

df_out = pd.DataFrame(
    {
        "true_label_id": labels_np,
        "true_label": true_label_str,
        "pred_label_id": ensemble_preds,
        "pred_label": pred_label_str,
    }
)
df_out.to_csv(out_path, index=False)
print(f"Saved ensemble predictions to: {out_path}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,1.128595,0.608696,0.465801
2,No log,0.971670,0.591304,0.609191
3,1.092900,0.929540,0.715942,0.699746
4,1.092900,1.001110,0.675362,0.653081
5,1.092900,1.062161,0.698551,0.667459
6,0.762600,1.064228,0.736232,0.701412
7,0.762600,1.247992,0.707246,0.662020
8,0.590400,1.167956,0.727536,0.694583
9,0.590400,1.208794,0.718841,0.683166
10,0.590400,1.204805,0.727536,0.697142



Evaluating microsoft/deberta-v3-large on held-out Task 1 test set...


microsoft/deberta-v3-large Test macro-F1: 0.6695
microsoft/deberta-v3-large Test accuracy: 0.711
microsoft/deberta-v3-large Test loss: 1.1196

==== Building RoBERTa + DeBERTa Ensemble on Task 1 test set ====
Ensemble (RoBERTa + DeBERTa) Test accuracy: 0.6948
Ensemble (RoBERTa + DeBERTa) Test macro-F1: 0.6501
Saved ensemble predictions to: deberta_roberta_task1_ensemble/task1_ensemble_predictions.csv
